In [ ]:
from flask import Flask, request, render_template
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [ ]:
app = Flask(__name__)

# Load the saved models
with open('Models/CNN.pkl', 'rb') as f:
    cnn_model = pickle.load(f)

with open('Models/VotingClassifier.pkl', 'rb') as f:
    sgd_model = pickle.load(f)

# Load the tokenizer and vectorizer
with open('Models/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

with open('Models/vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        text = request.form['text']
        model_name = request.form['model']
        result = classify_text(text, model_name)
        return render_template('index.html', result=result)
    return render_template('index.html', result=None)

def preprocess_text(text, model_name):
    if model_name == 'CNN':
        sequence = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=100)
        return sequence
    elif model_name == 'VotingClassifier':
        text_vectorized = vectorizer.transform([text])
        return text_vectorized

def classify_text(text, model_name):
    preprocessed_text = preprocess_text(text, model_name)
    if model_name == 'CNN':
        model = cnn_model
        prediction = model.predict(preprocessed_text)[0][0]
    elif model_name == 'VotingClassifier':
        model = sgd_model
        prediction = model.predict_proba(preprocessed_text)[0][1]
    else:
        return 'Invalid model selected'
    
    if prediction > 0.5:
        return "Text classified as AI."
    else:
        return "Text classified as human."

if __name__ == '__main__':
    app.run()